# Notebook Imports

In [56]:
import pandas as pd
import numpy as np

# Constants

In [103]:
VOCAB_SIZE = 2500


TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'

# Read and load features from .txt Files into NumPy array

In [58]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter = ' ', dtype = int)

In [59]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter = ' ', dtype = int)

In [60]:
#To get an idea of the size of this data, lets print the number of rows in this array
#We have done this with a dataframe now we do this using an array
print('Nr of rows in the training file:', sparse_train_data.shape[0])
print('Nr of rows in the testing file:', sparse_test_data.shape[0])

Nr of rows in the training file: 265570
Nr of rows in the testing file: 110568


In [61]:
print('Nr of emails in training file:',np.unique(sparse_train_data[:, 0]).size )
print('Nr of emails in testing file:',np.unique(sparse_test_data[:, 0]).size)

Nr of emails in training file: 4014
Nr of emails in testing file: 1723


### How to crate an empty dataframe

In [62]:
column_names = ['DOC_ID'] + ['CATEGORY'] +  list(range(0, VOCAB_SIZE))
column_names[:5]
len(column_names)

2502

In [63]:
index_names = np.unique(sparse_train_data[:,0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [64]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value = 0, inplace = True)

In [65]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Create a Full Matrix from sparse Matrix

In [66]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a pandas dataframe. 
    Keyword arguments:
    sparse_matrix -- numpy array
    nr_words -- size of the vocabulary. Total number of tokens. 
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of the label (spam is 1, nonspam is 0). Default: 3rd column
    freq_idx -- position of occurrence of word in sparse matrix. Default: 4th column
    """
    column_names = ['DOC_ID'] + ['CATEGORY'] +  list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        # Basically alloting each value, and use declared variables in function defn so that changing them is easy
        #Keeping the i same we are going through the compelete row using each column name for each word and storing data
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence
        #Here we are filling the values into the empty data frame
    
    full_matrix.set_index('DOC_ID', inplace = True)
    return full_matrix

In [67]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

CPU times: user 6.35 s, sys: 39.9 ms, total: 6.39 s
Wall time: 6.42 s


In [68]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,2,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2,0,1,2,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2,0,2,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,3,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,2,4,2,3,1,...,0,0,0,0,0,0,0,0,0,0


# Training the Naive Bayes Model

## Calculating the Probability of Spam

In [69]:
full_train_data.CATEGORY.size

4014

In [70]:
full_train_data.CATEGORY.sum()

1249

In [71]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print('Probability of spam is', prob_spam)

#Very smart method for this, as category 1 caters to spam

Probability of spam is 0.3111609367214748


## Total Number of Words / Tokens

In [72]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
#just removing the categories column, and then generating a datafram
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,2,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,2,4,2,3,1,1,...,0,0,0,0,0,0,0,0,0,0


In [73]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4014,)

In [74]:
email_lengths[:5]
#Summing up row wise, to get total number of tokens in a particular email

DOC_ID
0     50
1     76
2     87
3     76
4    137
dtype: int64

In [75]:
total_wc = email_lengths.sum()
total_wc
#Summing up all the rows to get toal number of tokens we have

444959

## Number of Tokens in Spam & Ham Emails

In [76]:
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_lengths.shape

(1249,)

In [77]:
spam_wc = spam_lengths.sum()
spam_wc

193999

In [78]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_lengths.shape

(2765,)

In [79]:
nonspam_wc = ham_lengths.sum()
nonspam_wc

250960

In [80]:
#Confirming things
email_lengths.shape[0] - spam_lengths.shape[0] - ham_lengths.shape[0]

0

In [81]:
spam_wc + nonspam_wc - total_wc

0

In [82]:
print('Average nr of words in spam emails {:.0f}'.format(spam_wc / spam_lengths.shape[0]))
print('Average nr of words in ham emails {:.3f}'.format(nonspam_wc / ham_lengths.shape[0]))
#THIS 0 AND 3 control the numbers after decimal points

Average nr of words in spam emails 155
Average nr of words in ham emails 90.763


## Summing the Tokens Occuring in Spam

In [83]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,2,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,2,4,2,3,1,1,...,0,0,0,0,0,0,0,0,0,0


In [84]:
train_spam_tokens.tail()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1885,2,0,0,3,2,2,2,2,1,0,...,0,0,0,0,0,0,0,0,0,0
1887,0,0,2,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,2,0,1,0,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1890,3,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1895,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
#Summing in columns
summed_spam_tokens = train_spam_tokens.sum(axis=0)+1
#adding 1 to make all calulation as non 0 while applying bayes
#Easy to do and doesnt change anything much
summed_spam_tokens

0       1781
1        968
2       1352
3       2095
4       1359
        ... 
2495      21
2496      17
2497       8
2498      14
2499      10
Length: 2500, dtype: int64

## Summing the Tokens Occuring in Ham

In [97]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]


In [98]:
summed_ham_tokens = train_ham_tokens.sum(axis=0)+1
summed_ham_tokens

0       5274
1       2501
2       2039
3        945
4       1597
        ... 
2495      12
2496      15
2497      21
2498      16
2499      21
Length: 2500, dtype: int64

## P(Token | Spam) - Probability that a Token Occurs given the Email is Spam

In [106]:
#Now as we added 1 in all words, now we need to increase the total word count as well before applying conditional probability
#This will minimize the error

prob_tokens_spam = summed_spam_tokens/(spam_wc+VOCAB_SIZE)
prob_tokens_spam.head()

0    0.009064
1    0.004926
2    0.006880
3    0.010662
4    0.006916
dtype: float64

In [107]:
prob_tokens_spam.sum()

1.0

## P(Token | Ham) - Probability that a Token Occurs given the Email is Nonspam

In [108]:
prob_tokens_nonspam = summed_ham_tokens / (nonspam_wc + VOCAB_SIZE)
prob_tokens_nonspam.sum()

1.0

# P(Token) - Probability that Token Occurs 

In [109]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc

# Save the training model

In [110]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

# Prepare Test Data

In [111]:
%%time
full_test_data = make_full_matrix(sparse_test_data, VOCAB_SIZE)

CPU times: user 3.12 s, sys: 104 ms, total: 3.22 s
Wall time: 4.05 s


In [115]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [116]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)